### Generate stimulus:

In [107]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActsX = 5
nActsY = 5
nWeights = 3
nOutsX = nActsX - nWeights + 1

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActsX,nActsY))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

print(f'PE cluster must be {nOutsX} PEs wide and {nWeights} PEs high ')

PE cluster must be 3 PEs wide and 3 PEs high 


In [108]:
a

array([[-4,  2,  0, -1,  2],
       [-2, -1,  1,  2,  1],
       [-4,  2,  1, -1, -1],
       [-3,  2,  2, -1,  2],
       [ 1,  2, -3, -3, -3]])

In [109]:
w

array([[ 1, -4,  0],
       [-4, -1,  0],
       [ 0, -2, -3]])

In [110]:
o

array([[-10,   6,   3],
       [  6, -15, -14],
       [  3,   3,  13]])

### Prepare scan chains for multicast ID

In [111]:
ids_acts = np.zeros((nOutsX+1,nWeights))
for x,idx in enumerate(ids_acts):
    for y,id in enumerate(idx):
        ids_acts[x][y] = x + y
ids_acts[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_acts

array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.],
       [0., 0., 0.]])

In [112]:
ids_weights = np.zeros((nOutsX+1,nWeights))
for x,idx in enumerate(ids_weights):
    for y,id in enumerate(idx):
        ids_weights[x][y] = x
ids_weights[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_weights

array([[0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 2.],
       [0., 0., 0.]])

In [113]:
tag_order_acts = []
for i in range(nActsX):
    tag_order_acts.append([0,i])
tag_order_acts = np.array(tag_order_acts)

tag_order_weights = []
for i in range(nWeights):
    tag_order_weights.append([0,i])
tag_order_weights = np.array(tag_order_weights)

### Save stimulus:

In [114]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
np.savetxt(path + '/a.txt',a,'%i')
np.savetxt(path + '/w.txt',w,'%i')
np.savetxt(path + '/o.txt',o,'%i')
np.savetxt(path + '/ids_acts.txt',ids_acts,'%i')
np.savetxt(path + '/ids_weights.txt',ids_weights,'%i')
np.savetxt(path + '/tag_order_acts.txt',tag_order_acts,'%i')
np.savetxt(path + '/tag_order_weights.txt',tag_order_weights,'%i')

### Have a look at existing stimulus:

In [115]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  

### Row-stationary Forensics

In [116]:
osize = nActsX-nWeights+1
sequence_array = [[
    (0, 0), # 0,0
    (1, 1), # 
    (2, 2),
    ],[
    (0, 1),
    (1, 2),
    (2, 3),
    ],[
    (0, 2),
    (1, 3),
    (2, 4)
    ]
]
ps = []
psr = []

for sequence in sequence_array:
    psr = []
    for wrow,arow in sequence:
        psr.append(np.convolve(a[arow],w[wrow][::-1],'valid'))   
    ps.append(psr)

np.array(ps)

# Expected PE partial sums:

array([[[-12.,   2.,   4.],
        [  9.,   3.,  -6.],
        [ -7.,   1.,   5.]],

       [[  2.,  -5.,  -7.],
        [ 14.,  -9.,  -3.],
        [-10.,  -1.,  -4.]],

       [[-12.,  -2.,   5.],
        [ 10., -10.,  -7.],
        [  5.,  15.,  15.]]])

In [117]:
ress = []
for psr in ps:
    res = psr.copy()
    for i in range(1,len(res)):
        res[i] += res[i-1]
    ress.append(res)
np.array(ress)

# Expected partial sum outputs in systolic passing:

array([[[-12.,   2.,   4.],
        [ -3.,   5.,  -2.],
        [-10.,   6.,   3.]],

       [[  2.,  -5.,  -7.],
        [ 16., -14., -10.],
        [  6., -15., -14.]],

       [[-12.,  -2.,   5.],
        [ -2., -12.,  -2.],
        [  3.,   3.,  13.]]])